In [5]:
import pandas as pd
import folium
from folium import plugins
from folium.plugins import HeatMap


def showmap(myDf, param):
    if param == "relative_scholarship":
        label = "Scholarship"
    elif param == "paidfee":
        label = "Fee"
    elif param == "perc_intern":
        label = "International students (%)"
    lats_longs_weight = list(map(list, zip(myDf["lat"], myDf["lon"], round(myDf[param], 2))))
    map_obj = folium.Map(location = [43, 11], zoom_start = 6)
    heatmap_layer = HeatMap(lats_longs_weight, name = label).add_to(map_obj)
    

    group1 = folium.FeatureGroup(name='Markers')
    map_obj .add_child(group1)

    for idx, row in myDf.iterrows():
        textmarker = str(row["uni"] + "\n\n"+ "{0}: ".format(label) + str(round(row[param], 2)))
        folium.Marker(
            [row["lat"], row["lon"]], 
            popup= textmarker,
            icon=folium.Icon(color='red', icon='graduation-cap', prefix='fa') 
            ).add_to(group1)
    
    # add layer control to map (allows layers to be turned on or off)
    folium.LayerControl(collapsed=False).add_to(map_obj)
    
    return map_obj

geoDf = pd.read_csv("../../../initinere/data/output/geocordinatesuni.csv")

showmap(geoDf, "perc_intern").save("../../../initinere/assets/leafletgeomap/heat_map_perc.html")
showmap(geoDf, "paidfee").save("../../../initinere/assets/leafletgeomap/heat_map_paidfee.html")
showmap(geoDf, "relative_scholarship").save("../../../initinere/assets/leafletgeomap/heat_map_dsu.html")
showmap(geoDf, "perc_intern")